In [134]:
# Python Version:
#!python --version

# Pip:
#!pip install --disable-pip-version-check -q pip --upgrade > /dev/null
#!pip install --disable-pip-version-check -q wrapt --upgrade > /dev/null

# AWS CLI:
#!pip install --disable-pip-version-check -q awscli==1.18.216 boto3==1.16.56 botocore==1.19.56

# AWS SageMaker:
#!pip install --disable-pip-version-check -q sagemaker==2.29.0
#!pip install --disable-pip-version-check -q smdebug==1.0.1
#!pip install --disable-pip-version-check -q sagemaker-experiments==0.1.26

# AWS Redshift:
#!pip install --disable-pip-version-check -q SQLAlchemy==1.3.22
#!pip install --disable-pip-version-check -q psycopg2-binary==2.9.1

# Zip:
#!conda install -y zip

# Pip List:
#!pip list

In [12]:
# IAM Roles:

import boto3
import sagemaker
import time
from time import gmtime, strftime

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()
region = boto3.Session().region_name

from botocore.config import Config

config = Config(retries={"max_attempts": 10, "mode": "adaptive"})

iam = boto3.client("iam", config=config)

# Getting Our Role Name of LabRole:
role_name = role.split("/")[-1]

print("Role name: {}".format(role_name))

# Checking permissions of our role:
admin = False
post_policies = iam.list_attached_role_policies(RoleName=role_name)["AttachedPolicies"]
for post_policy in post_policies:
    if post_policy["PolicyName"] == "AdministratorAccess":
        admin = True
        setup_iam_roles_passed = True
        print("[OK]")
        break
    else:
        print("*************** [ERROR] SageMakerExecutionRole needs the AdministratorAccess policy attached. *****************")

Role name: LabRole
*************** [ERROR] SageMakerExecutionRole needs the AdministratorAccess policy attached. *****************
*************** [ERROR] SageMakerExecutionRole needs the AdministratorAccess policy attached. *****************
*************** [ERROR] SageMakerExecutionRole needs the AdministratorAccess policy attached. *****************
*************** [ERROR] SageMakerExecutionRole needs the AdministratorAccess policy attached. *****************
[OK]


In [13]:
# Start with Creation of the S3 Bucket

s3 = boto3.Session().client(service_name="s3", region_name=region)

In [14]:
setup_s3_bucket_passed = False

In [15]:
print("Default bucket: {}".format(bucket))

Default bucket: sagemaker-us-east-1-669765645205


In [16]:
%%bash

aws s3 ls s3://${bucket}/

2022-03-02 23:59:17 sagemaker-studio-669765645205-0l5jebx5msd
2022-03-03 00:39:00 sagemaker-us-east-1-669765645205


In [17]:
from botocore.client import ClientError

response = None

try:
    response = s3.head_bucket(Bucket=bucket)
    print(response)
    setup_s3_bucket_passed = True
except ClientError as e:
    print("[ERROR] Cannot find bucket {} in {} due to {}.".format(bucket, response, e))

{'ResponseMetadata': {'RequestId': 'KWVFX296ZTNFX03X', 'HostId': 'Y8ic4LmHMU1MyCqCxZVVQ99Ir0IrDggNGWyHp70c2lnqu3WNhcXFiyrlCG/TX6UvH6Di2QEAAw4=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'Y8ic4LmHMU1MyCqCxZVVQ99Ir0IrDggNGWyHp70c2lnqu3WNhcXFiyrlCG/TX6UvH6Di2QEAAw4=', 'x-amz-request-id': 'KWVFX296ZTNFX03X', 'date': 'Thu, 24 Mar 2022 01:08:44 GMT', 'x-amz-bucket-region': 'us-east-1', 'x-amz-access-point-alias': 'false', 'content-type': 'application/xml', 'server': 'AmazonS3'}, 'RetryAttempts': 0}}


In [79]:
#s3.list_objects(Bucket=bucket)

In [87]:
s3_bucket = pd.DataFrame.from_dict(s3.list_objects(Bucket=bucket)['Contents'])
s3_bucket.head()

,Key,LastModified,ETag,Size,StorageClass,Owner
0,Raw Data/SDGE/Electric/SDGE-ELEC-2012-Q1.csv,2022-03-23 23:28:57+00:00,"""0070e4176e266ce0618f23246cf009ea""",45132,STANDARD,{'DisplayName': 'awslabsc0w3920716t1646245937'...
1,Raw Data/SDGE/Electric/SDGE-ELEC-2012-Q2.csv,2022-03-23 23:28:58+00:00,"""42fe5a7a241a495141c14ca125ccedef""",46929,STANDARD,{'DisplayName': 'awslabsc0w3920716t1646245937'...
2,Raw Data/SDGE/Electric/SDGE-ELEC-2012-Q3.csv,2022-03-23 23:28:58+00:00,"""f5c3fa472acbad6784ba52378470090d""",47229,STANDARD,{'DisplayName': 'awslabsc0w3920716t1646245937'...
3,Raw Data/SDGE/Electric/SDGE-ELEC-2012-Q4.csv,2022-03-23 23:28:59+00:00,"""2f4e38aa66cd2a55950432ef10687aa5""",48962,STANDARD,{'DisplayName': 'awslabsc0w3920716t1646245937'...
4,Raw Data/SDGE/Electric/SDGE-ELEC-2013-Q1.csv,2022-03-23 23:28:59+00:00,"""b7fee21186a49daaa4c39e4bcee5ca38""",47242,STANDARD,{'DisplayName': 'awslabsc0w3920716t1646245937'...


In [85]:
#list(dir(s3))[45:]

In [130]:
path = '/root/ADS_508/Raw_Data'
os.mkdir(path)
os.chdir(path)

In [133]:
for i in list(range(0,len(s3_bucket),1)):
    file_path = s3_bucket.loc[i,'Key']
    
    
    file_path = s3_bucket.loc[i,'Key']
    filename = file_path.split('/')[::-1][0]
    
    s3.download_file(Bucket=bucket, Filename=filename, Key=file_path)           
    
    print(filename)
    print(file_path)